In [1]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

model = init_chat_model("google_genai:gemini-2.5-flash-lite")

c:\Users\thaia\Desktop\Financial Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [3]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [4]:
import json

paths = [r"C:\Users\thaia\Desktop\Financial Chatbot\data\yuanta_chung_khoan_20251219_203757.json",\
    r'C:\Users\thaia\Desktop\Financial Chatbot\data\yuanta_chung_khoan_20251219_203722.json']

data = []
for path in paths:
	with open(path, "r", encoding="utf-8") as f:
		data += json.load(f)


In [5]:
! pip install -qU langchain-qdrant

In [6]:
from langchain_core.documents import Document

docs = []

for item in data:
    # Bạn chọn field muốn cho vào page_content
    text = f"{item.get('title', '')}\n\n{item.get('content', '')}"

    doc = Document(
        page_content=text,
        metadata={
            "title": item.get("title"),
            "content": item.get("content"),
        }
    )
    docs.append(doc)

print("Số document:", len(docs))

Số document: 58


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 201 sub-documents.


In [8]:
! pip install faiss-cpu

In [9]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

['3775d445-55e3-4e09-b2f8-e9e21eb182d3', '13e2c999-158d-4592-ad89-74f91fd328cd', 'd83684c2-b31d-48ff-8ad8-ee2cfb8f9b1f']


In [11]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [16]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [17]:
query = (
    "FPT là công ty gì"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

FPT là công ty gì
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (a83aca88-79be-445c-8ded-9c54867b9f01)
 Call ID: a83aca88-79be-445c-8ded-9c54867b9f01
  Args:
    query: FPT là công ty gì
================================= Tool Message =================================
Name: retrieve_context

Source: {'title': 'Cổ tức là gì', 'content': ['1. Cổ tức là gì?', '1.0.1 Cổ tức là khoản lợi nhuận mà công ty chia cho cổ đông theo định kỳ hàng năm, thường dưới dạng tiền mặt, cổ tức bằng cổ phiếu, hoặc cổ tức dưới dạng tài sản. Lưu ý là công ty sẽ thanh toán cổ tức cho cổ đông khi công ty có lợi nhuận, một số trường hợp công ty có lợi nhuận nhưng vẫn không chia cổ tức vì mục đích tái đầu tư khoản lợi nhuận này hoặc là để gia tăng giá trị công ty', '1.0.2 Ví dụ ở Mỹ cổ phiếu của nhà đầu tư huyền thoại Warrent Buffet là cổ phiếu Berkshire\xa0 Hathaway đã

In [1]:
import psycopg2

In [28]:
conn = psycopg2.connect(
    host="localhost",
    port=5430,
    user="admin",
    password="admin123",
    database="postgres"
)


In [38]:
cur = conn.cursor()
sql = 'Select * from stock.company'

In [39]:
cur.execute(sql)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
